# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [116]:
# import libraries

import numpy as np
import pandas as pd
import nltk
import sqlite3
import sqlalchemy
import sys
import io


from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
from sklearn.base import BaseEstimator,TransformerMixin

from xgboost import XGBClassifier

In [61]:
# Connect to known databases
conn_chrismoetable = sqlite3.connect('chrismoetable.db')
conn_disasterresponse = sqlite3.connect('DisasterResponse.db')
conn_insertdb = sqlite3.connect('InsertDatabaseName.db')

def list_tables(connection):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return connection.execute(query).fetchall()

# List tables for each database
chrismoetable_tables = list_tables(conn_chrismoetable)
disasterresponse_tables = list_tables(conn_disasterresponse)
insertdatabase_tables = list_tables(conn_insertdb)

# Close the connections
conn_chrismoetable.close()
conn_disasterresponse.close()
conn_insertdb.close()

# Output the results
print(chrismoetable_tables)
print(disasterresponse_tables)
print(insertdatabase_tables)

[]
[('Message',)]
[('process_data_table',), ('final',), ('chrismo',)]


In [63]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('chrismo', con=engine)

In [64]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
df.isna().sum()

id                            0
message                       0
original                  16140
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [69]:
df.nunique()

id                        26180
message                   26177
original                   9630
genre                         3
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
child_alone                   1
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_in

In [71]:
X = df['message']
Y = df.iloc[:,4:]

In [73]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [75]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
print(X.shape)
print(Y.shape)

(26386,)
(26386, 36)


In [79]:
print(X.isna().sum().sum())
print(Y.isna().sum().sum())

0
0


### 2. Write a tokenization function to process your text data

In [82]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [95]:
print(X.shape)
print(Y.shape)

(26386,)
(26386, 36)


In [97]:
X.shape

(26386,)

In [99]:
Y.shape

(26386, 36)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size = 0.25)

pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

In [ ]:
def capture_classification_report(y_true, y_pred):
    # Redirect stdout to capture the print output
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    try:
        print(classification_report(y_true, y_pred))
    finally:
        sys.stdout = old_stdout
    return buffer.getvalue()

# Iterate through each column in the target dataframe Y
for column in Y.columns:
    report = capture_classification_report(y_test[column], y_pred[:, Y.columns.get_loc(column)])
    print(f"Category: {column}\n")
    print(report)
    print("\n" + "="*80 + "\n")

In [108]:
# List of models to evaluate, including XGBoost
models = {
    'Random Forest': RandomForestClassifier(n_estimators=400, max_depth=32, min_samples_split=10, n_jobs=-1),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

In [112]:
pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
        ])),
        ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=400, max_depth=32, min_samples_split=10, n_jobs=-1)))
    ])

In [120]:
pipeline.fit(X_train, Y_train)

/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('count_vectorizer',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x32d8a4e00>)),
                                                                 ('tfidf_transformer',
                                                                  TfidfTransformer())]))])),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=32,
                                                                        min_samples_split=10,
                                                                        n_estimators=400,
                                                                        n_jobs=-1)))])

In [140]:
print(X.shape)
print(Y.shape)

(26386,)
(26386, 36)


In [134]:
y_predictions = pipeline.predict(X_test)

In [135]:
y_predictions_df = pd.DataFrame(y_predictions, columns = y_test.columns)
y_predictions_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [136]:
for column in y_test.columns:
    print('....................................................\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_predictions_df[column]))

....................................................

FEATURE: related



ValueError: Found input variables with inconsistent numbers of samples: [6597, 5278]

In [124]:
def model_report(y_test, y_pred):
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [126]:
model_report(y_test, y_pred)

Feature 1: related


ValueError: Found input variables with inconsistent numbers of samples: [6597, 5278]

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Iterate over each model
for name, model in models.items():
    # Create a pipeline for each model
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
        ])),
        ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=400, max_depth=32, min_samples_split=10, n_jobs=-1)))
    ])
    
    # Fit the model on the training data
    pipeline.fit(X_train, Y_train)
    
    # Predict on the test data
    y_pred = pipeline.predict(X_test)
    
    # Output classification report for each model
    print(f"Classification Report for {name}:\n")
    print(classification_report(Y_test, y_pred))
    print("\n" + "="*60 + "\n")

In [ ]:
acc = accuracy_score(y_test, y_pred)
print(acc)

accuracy = accuracy_score(y_test, y_pred_grid)
precision = precision_score(y_test, y_pred_grid, average='micro')
recall = recall_score(y_test, y_pred_grid, average='micro')


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def capture_classification_report(y_true, y_pred):
    # Redirect stdout to capture the print output
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    try:
        print(classification_report(y_true, y_pred))
    finally:
        sys.stdout = old_stdout
    return buffer.getvalue()

# Iterate through each column in the target dataframe Y
for column in Y.columns:
    report = capture_classification_report(y_test[column], y_pred[:, Y.columns.get_loc(column)])
    print(f"Category: {column}\n")
    print(report)
    print("\n" + "="*80 + "\n")

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
                'clf__estimator__n_estimators': [100, 200],
                'clf__estimator__max_depth': [3, 6, 10],
                'clf__estimator__learning_rate': [0.01, 0.1]
    
              }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=1, cv=3)

In [ ]:
print(cv)

In [ ]:
# Fit the grid search
cv.fit(X_train, y_train)

In [ ]:
# Best parameters
best_params = cv.best_params_
#preditcitng using best estimator 
print("Best parameters found: ", best_params)

In [ ]:
y_pred_grid = cv.best_estimator_.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
accuracy = accuracy_score(y_test, y_pred_grid)
precision = precision_score(y_test, y_pred_grid, average='micro')
recall = recall_score(y_test, y_pred_grid, average='micro')


In [ ]:
print(f"Accuracy: {accuracy}")

In [ ]:
print(f"Precision: {precision}")

In [ ]:
print(f"Recall: {recall}")

In [ ]:
print("\nClassification Report:\n", classification_report(y_test, y_pred_grid))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.